In [ ]:
""" 
    5. Sweep the Parameters ---- Prep Stage
"""
import pyvisa
import numpy as np
from math import log10
from math import gcd
from pathlib import Path
import shutil
from time import sleep
import csv
import os
import datetime
import matplotlib.pyplot as plt
from sympy import primerange, nextprime
import KS33600A_Control as FG
import KSE36311A_Control as PS
import KSMSOX3104G as OSC
PS_INSTRUMENT_ID = 'USB0::0x2A8D::0x8F01::CN61180122::0::INSTR'
FG_INSTRUMENT_ID = 'USB0::0x0957::0x5707::MY59004733::0::INSTR'
OSC_INSTRUMENT_ID = 'USB0::0x2A8D::0x1770::MY63420204::0::INSTR'

def round_to_next_odd(number):
    # Check if the number is odd, if so return the number
    if number % 2 != 0:
        return number
    else:
        return number + 1
    
def generate_coherent_freq(fdsample, fdin, totalpoints,sigfig):
    fdbin = fdsample/totalpoints
    fcbin = round(fdbin,sigfig)
    fcsample = round(fcbin*totalpoints, sigfig)
    Ndwindow = fdin/fcsample*totalpoints
    tdwindow = totalpoints/fdsample
    ncwindow = round_to_next_odd(int(Ndwindow)+1)
    fcin = round(fcsample*ncwindow/totalpoints, sigfig)
    return fcsample, fcin

def get_corhent_sampling(fs,Mpoints,fin_in):
    # Parameters
    # fs = 1e6                # Sampling frequency
    # Mpoints = 2**13         # Number of points
    tstop = Mpoints / fs    # Total time

    fin_set = fin_in          # Target frequency

    # Number of Cycles, i.e., Nbins, also called Nwindow
    Nbins = fin_set * Mpoints / fs

    # Find nearest prime numbers used in correlated sampling
    primes_list = list(primerange(1, int(Nbins) + 1))
    primes_list.append(nextprime(Nbins))  # Append the next prime after Nbins
    primeNums_nearest = [primes_list[-2], primes_list[-1]]

    # Calculate errors for the nearest primes
    fin_error = [fs * primeNums_nearest[0] / Mpoints - fin_set,
                fs * primeNums_nearest[1] / Mpoints - fin_set]

    # Find the nearest prime that minimizes error
    fh_error_temp = min(err for err in fin_error)
    Index = fin_error.index(fh_error_temp)
    N = primeNums_nearest[Index]

    # Calculate final frequency
    fin = fs * N / Mpoints
    return fin


fsample = 1e6
start_freq = 1e3
totalpoints = 2**13
sigfig = 4
start = log10(start_freq )
stop_freq = 100e3
stop = log10(stop_freq)
frequencies_SNR = np.logspace(start, stop, num=20, endpoint=True, base=10.0, dtype=None, axis=0)
frequencies_SNR = np.insert(frequencies_SNR, 0, 2e3)
frequencies_SNR = np.append(frequencies_SNR,250e3)
clk_array = np.zeros(len(frequencies_SNR))
coherent_freq = np.zeros(len(frequencies_SNR))
# for index, freq in enumerate(frequencies_SNR):
#     [clk_temp,freq_temp] = generate_coherent_freq(fsample/1e6,freq/1e6,totalpoints,sigfig)
#     coherent_freq[index] = freq_temp
#     clk_array[index] = clk_temp
for index, freq in enumerate(frequencies_SNR):
    freq_temp = get_corhent_sampling(fsample,totalpoints,freq)
    coherent_freq[index] = freq_temp
    clk_array[index] = 4*freq_temp


"""Instrument ID Setup"""
fg = FG.FuncGenConnect(FG_INSTRUMENT_ID)

osc = OSC.Oscilloscope_Connect(OSC_INSTRUMENT_ID)

ps = PS.PowerSupply_Connect(PS_INSTRUMENT_ID)


""" 
    1. Power Supply Control
"""

# Set channel 1 to 2.5 V, 0.2A
PS.Voltage_Setup(ps,1,2.5)
PS.Current_Setup(ps,1,0.2)

# Set channel 2 to 5 V, 0.2A
PS.Voltage_Setup(ps,2,5)
PS.Current_Setup(ps,2,0.2)

# Set channel 3 to -5 V, 0.2A
PS.Voltage_Setup(ps,3,5)
PS.Current_Setup(ps,3,0.2)

PS.Output_OFF(ps,1)
PS.Output_OFF(ps,2)
PS.Output_OFF(ps,3)

"""
    Setup the Oscilloscope
"""
# Free Run the oscilloscope
OSC.Oscilloscope_RUN(osc)
# Bundle the D0-8 and D11 to BUS1
OSC.Oscilloscope_SetBUS(osc)
# Erase the digital display first
# Then turn on D0-8 and D11
OSC.DigitalDisplay_ON(osc,1)
# Set the trigger source to D11
OSC.Oscilloscope_Trigger_Dchan(osc,11)

# Oscilloscope_Trigger_External(osc)

# Set the threshold voltage
digi_th = 1.8
OSC.SetDigital_Threshold(osc,digi_th)
# Set the timebase to 1ms/div
OSC.Oscilloscope_TimeBase(osc, 5/1e3)

# Setup the CLK coming from the Oscilloscope
OSC.Oscilloscope_WGen_Square_OFF(osc)
clk_freq = clk_array[0]
clk_amp = 2.5
clk_offset = 1.25
clk_duty = 50
OSC.Oscilloscope_WGen_Square(osc,clk_freq,clk_amp,clk_offset,clk_duty)
OSC.Oscilloscope_WGen_Square_OFF(osc)
sleep(3)

"""
    Setup the FunctionGenerator
"""
FG.UnitVpp_setup(fg,1)
FG.Load__setup(fg,1,1)

FG.UnitVpp_setup(fg,2)
FG.Load__setup(fg,2,1)
freq = coherent_freq[0]
sin_amp = 1
sin_offset = 0.5

FG.Sin_setup(fg,1,freq,sin_amp,sin_offset)
FG.Sin_setup(fg,2,freq,sin_amp,sin_offset)
# Invert the second channel
FG.Polarity_invert(fg,2,1)
# Sync the phase
FG.Sync_phase(fg)

FG.output_OFF(fg,1)
FG.output_OFF(fg,2)
OSC.Oscilloscope_WGen_Square_ON(osc)

In [ ]:
PS.Output_ON(ps,1)
PS.Output_ON(ps,2)
PS.Output_ON(ps,3)
sleep(5)

for index, freq_var in enumerate(coherent_freq):
    # Sin Input parameters
    sin_freq = freq_var
    FG.Sin_setup(fg,1,sin_freq,sin_amp,sin_offset)
    FG.Sin_setup(fg,2,sin_freq,sin_amp,sin_offset)
    FG.Polarity_invert(fg,2,1)
    FG.Sync_phase(fg)
    print("frequency @" + str(sin_freq))

    # Turn on THE CLK, and sleep 3
    OSC.Oscilloscope_WGen_Square_ON(osc)
    FG.Sync_phase(fg)
    sleep(3)
    OSC.Oscilloscope_Single(osc)
    FG.Sync_phase(fg)
    sleep(3)
    OSC.Oscilloscope_Single(osc)
    FG.Sync_phase(fg)
    sleep(3)
    ################## Save Data from Oscilloscope
    [preamble, data] = OSC.Save_waveform(osc,"BUS1",)
    combined_string = ''.join(data)
    preamble_string = ''.join(preamble)

    # Step 2: Split the string by the commas
    rows = combined_string.split(',')
    row_preamble = preamble_string.split(',')
    new_data = []
    i = 0
    for raw in rows[1:]:
        new_data.append(int(raw, 16))
        i = i+1

    # Folder Path
    folder_path = r"C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\CML-Data"  # Change to your desired path

    ### !!!!!!!!!change folder name as needed!!!!!!!!!!!!!
    Board_NO = 4
    folder_properties = "BYPASS-Freq_1k_250k-SIN_"+str(int(sin_amp*1000))+"mV-CLK_"+str(int(clk_freq/1e6)) + "MHz_Duty" + str(int(clk_duty))+"%"
    folder_name = "12-2-SHIKAITB" + "Board"+str(Board_NO)+"_" + folder_properties
    file_path = os.path.join(folder_path, folder_name)
    os.makedirs(file_path, exist_ok=True)
    # Create the file
    filename = "SIN_"+str(sin_freq)+"Hz_" +".csv"
    file_path = os.path.join(file_path, filename)


    # Write the data to file and Save it
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each integer as a row
        for number in new_data:
            writer.writerow([number]) 
    print(f"Data saved to {file_path}")



    OSC.Oscilloscope_RUN(osc)
    OSC.Oscilloscope_WGen_Square_OFF(osc)
    FG.output_OFF(fg,2)
    FG.output_OFF(fg,1)
    sleep(3)


# Save the sweeping frequencies array to a .csv
file_path = os.path.join(folder_path, folder_name)
os.makedirs(file_path, exist_ok=True)
################## Create the file
filename = "freq_array.csv"
file_path = os.path.join(file_path, filename)
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write each integer as a row
    for number in coherent_freq:
        writer.writerow([number]) 

print(f"Data saved to {file_path}")

# Save the clk frequencies array to a .csv
file_path = os.path.join(folder_path, folder_name)
os.makedirs(file_path, exist_ok=True)
################## Create the file
filename = "clk_array.csv"
file_path = os.path.join(file_path, filename)
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write each integer as a row
    for number in clk_array:
        writer.writerow([number]) 

print(f"Data saved to {file_path}")

OSC.Oscilloscope_WGen_Square_OFF(osc)
FG.output_OFF(fg,1)
FG.output_OFF(fg,2)
PS.Output_OFF(ps,1)
PS.Output_OFF(ps,2)
PS.Output_OFF(ps,3)


In [8]:
"""
    6. Turn off All power supply and function generator, and    
"""

OSC.Oscilloscope_WGen_Square_OFF(osc)

# Power Supply channel 1,2,3 OFF
PS.Output_OFF(ps,1)
PS.Output_OFF(ps,2)
PS.Output_OFF(ps,3)

# Function Generator channel 1 and 2 OFF
FG.output_OFF(fg,1)
FG.output_OFF(fg,2)
osc.close()
fg.close()
ps.close()